# Advent of Code: Day 2

## Part One

> You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the 11 in Game 11: ...) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like 3 red, 5 green, 4 blue).
>
> For example, the record of a few games might look like this:
>
> ```
> Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
> Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
> Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
> Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
> Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
> ```
> In game 1, three sets of cubes are revealed from the bag (and then put back again). The first set is 3 blue cubes and 4 red cubes; the second set is 1 red cube, 2 green cubes, and 6 blue cubes; the third set is only 2 green cubes.
>
> The Elf would first like to know which games would have been possible if the bag contained only 12 red cubes, 13 green cubes, and 14 blue cubes?
>
> In the example above, games 1, 2, and 5 would have been possible if the bag had been loaded with that configuration. However, game 3 would have been impossible because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been impossible because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get 8.
>
> Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. What is the sum of the IDs of those games?



In [ ]:
from dataclasses import dataclass
import logging
import re
from typing import Dict, List, Tuple

In [ ]:
# Set up logging for Jupyter Notebook
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
@dataclass
class GameResult():
    red: int = 0
    green: int = 0
    blue: int = 0

    def __setitem__(self, key, value):
        if key not in ('red', 'green', 'blue'):
            raise ValueError(f'Key {key} not allowed.')
        if getattr(self, key) != 0:
            raise ValueError(f'Key {key} is set more than once')
        setattr(self, key, value)

    def __getitem__(self, key):
        return getattr(self, key)


def extract_game_id(s: str) -> int:
    re_pattern = r'Game (?P<game_id>\d+)'
    p = re.compile(re_pattern)
    m = p.search(s)
    if m is not None:
        return int(m.group('game_id'))
    else:
        return None


def parse_game_set_str(s: str) -> GameResult:
    re_pattern: str = r'(?P<count>\d+) (?P<color>red|green|blue)'
    p = re.compile(re_pattern)
    game_result: GameResult = GameResult()
    for color_result in s.split(','):
        m = p.search(color_result)
        color = m.group('color')
        count = int(m.group('count'))
        game_result[color] = count
    return game_result


def parse_game_str(s: str) -> List[GameResult]:
    game_results: List[GameResult] = []
    game_set_str_list: List[str] = s.split(';')
    for game_set_str in game_set_str_list:
        game_result: GameResult = parse_game_set_str(game_set_str)
        # logger.info(game_result)
        game_results.append(game_result)
    return game_results


def parse_line(line: str) -> Tuple[int, List[GameResult]]:
    # logger.debug(line)
    game_id_str, game_results_str = line.split(':')
    game_id = extract_game_id(game_id_str)
    # logger.info(f'game_id: {game_id}')
    game_results: List[GameResult] = parse_game_str(game_results_str)
    return game_id, game_results


def solution_part1():
    game_log: Dict[int, List[GameResult]] = dict()
    with open('20231202-input.txt', 'r') as f:
        for line in f.readlines():
            game_id, game_results = parse_line(line.strip())
            game_log[game_id] = game_results
    reference_game_result: GameResult = GameResult(
        red=12,
        green=13,
        blue=14,
    )
    possible_list: List[int] = []
    for game_id, game_results in game_log.items():
        max_game_result = GameResult(
            red=max([r['red'] for r in game_results]),
            green=max([r['green'] for r in game_results]),
            blue=max([r['blue'] for r in game_results]),
        )
        #logger.info(f'game_results: {game_results}')
        if max_game_result.red <= reference_game_result.red and max_game_result.green <= reference_game_result.green and max_game_result.blue <= reference_game_result.blue:
            logger.info(f'max_game_result: {max_game_result}')
            possible_list.append(game_id)
    logger.info(possible_list)
    return sum(possible_list)
        


solution_part1()

## Part Two

> The Elf says they've stopped producing snow because they aren't getting any water! He isn't sure why the water stopped; however, he can show you how to get to the water source to check it out for yourself. It's just up ahead!
>
> As you continue your walk, the Elf poses a second question: in each game you played, what is the fewest number of cubes of each color that could have been in the bag to make the game possible?
>
> Again consider the example games from earlier:
>
> ```
> Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
> Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
> Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
> Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
> Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
>```
>
> - In game 1, the game could have been played with as few as 4 red, 2 green, and 6 blue cubes. If any color had even one fewer cube, the game would have been impossible.
> - Game 2 could have been played with a minimum of 1 red, 3 green, and 4 blue cubes.
> - Game 3 must have been played with at least 20 red, 13 green, and 6 blue cubes.
> - Game 4 required at least 14 red, 3 green, and 15 blue cubes.
> - Game 5 needed no fewer than 6 red, 3 green, and 2 blue cubes in the bag.
>
> The power of a set of cubes is equal to the numbers of red, green, and blue cubes multiplied together. The power of the minimum set of cubes in game 1 is 48. In games 2-5 it was 12, 1560, 630, and 36, respectively. Adding up these five powers produces the sum 2286.
>
> For each game, find the minimum set of cubes that must have been present. What is the sum of the power of these sets?

In [ ]:
def solution_part2():
    game_log: Dict[int, List[GameResult]] = dict()
    with open('20231202-input.txt', 'r') as f:
        for line in f.readlines():
            game_id, game_results = parse_line(line.strip())
            game_log[game_id] = game_results
    game_power_list: List[int] = []
    for game_id, game_results in game_log.items():
        max_game_result = GameResult(
            red=max([r['red'] for r in game_results]),
            green=max([r['green'] for r in game_results]),
            blue=max([r['blue'] for r in game_results]),
        )
        game_power: int = max_game_result.red * max_game_result.green * max_game_result.blue
        game_power_list.append(game_power)
        logger.info(f'max_game_result: {max_game_result}')
        logger.info(f'game_power: {game_power}')
    return sum(game_power_list)


solution_part2()